In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
inputs = df.drop('Survived',axis='columns')
target = df['Survived']

In [4]:
from sklearn.preprocessing import LabelEncoder
le_sex = LabelEncoder()

In [5]:
inputs['sex_n'] = le_sex.fit_transform(inputs['Sex'])

In [6]:
inputs

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,sex_n
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1


In [11]:
inputs_n = inputs.drop(['PassengerId','Name','Age','Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked'],axis='columns')
inputs_n

,Pclass,Fare,sex_n
0,3,7.2500,1
1,1,71.2833,0
2,3,7.9250,0
3,1,53.1000,0
4,3,8.0500,1
...,...,...,...
886,2,13.0000,1
887,1,30.0000,0
888,3,23.4500,0
889,1,30.0000,1


In [12]:
target

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [13]:
from sklearn import tree
model = tree.DecisionTreeClassifier()

In [15]:
model.fit(inputs_n, target)

DecisionTreeClassifier()

In [16]:
model.score(inputs_n,target)

0.9046015712682379

In [23]:
model.fit(inputs_n.values, target.values)
data_pred = model.predict(inputs_n.values)

## Predict score if Pclass = 3, fare = 7.2500, sex = male (1)

array([0]) means did not survive. array([1]) means did survive

In [25]:
model.predict([[2,7.2500,1]])

array([0])

## Predict score if Pclass = 1, fare = 71, sex = female (0)

In [28]:
model.predict([[1,71.2833,0]])

array([1])